In [1]:
import torch
import transformers
import pandas as pd
import numpy as np

from sklearn import model_selection, metrics

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\naiti\AppData\Roaming\Python\Python311\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [2]:
"""
* Dataset class
* Model
* Trainer - training arguments
"""

'\n* Dataset class\n* Model\n* Trainer - training arguments\n'

In [3]:
config = {
    "max_length": 360,
    "model_path": "microsoft/xtremedistil-l6-h256-uncased",
    
    "output_dir": "./my-model",
    "train_batch_size": 64,
    "valid_batch_size": 64,
    "learning_rate": 3e-5,
    "epochs": 3,
    
    "debug": True,
}

In [4]:
class TextDataset:
    
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        enc = ...
        
        return {
            "input_ids": torch.tensor(enc["input_ids"]),
            "attention_mask": torch.tensor(enc["attention_mask"]),
            "label": torch.tensor(row["label"]),
        }

In [5]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv").rename(columns={"review": "text"})

id2label = {0: "negative", 1: "positive"}
label2id = {label: id_ for id_, label in id2label.items()}

df["label"] = df["sentiment"].map(label2id)

if config["debug"]:
    print("DEBUG MODE!")
    df = df.sample(10_000, random_state=123)

print(df.shape)
df.head()

DEBUG MODE!
(10000, 3)


,text,sentiment,label
11872,"This movie was beyond awful, it was a pimple o...",negative,0
40828,As of this writing John Carpenter's 'Halloween...,positive,1
36400,I must admit a slight disappointment with this...,positive,1
5166,Oh dear! The BBC is not about to be knocked of...,negative,0
30273,its a totally average film with a few semi-alr...,negative,0


In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(config["model_path"])

In [7]:
train, valid = model_selection.train_test_split(
    df,
    test_size=0.2,
    random_state=1123,
    shuffle=True,
    stratify=df["label"]
)

In [8]:
train_ds = TextDataset(train)
valid_ds = TextDataset(valid)

In [9]:
valid_ds[0]

{'input_ids': tensor([  101,  6240, 18510,  2573,  2049,  2126,  2046, 11450,  1010,  2066,
          2053,  2060,  3185,  1012,  2926,  2076,  2621,  1012,  3251,  2009,
          1005,  1055,  1996,  2299,  2055,  1996, 25022,  3215,  1006, 17220,
          2015,  1999,  2731,  1007,  2030,  1996,  3013,  1011, 12482,  2030,
          1996, 28676, 18254,  1996, 10442,  2030,  2009,  2074,  2987,  1005,
          1056,  3043,  4613,  1012,  1012,  1012,  2009, 16949,  2039,   999,
          3532, 11021,  1013, 22294,  2100,  1010,  2040,  4282,  2073,  2002,
          1005,  1040,  5256,  2039,  2279,   999,  1029,   999,  2107,  1037,
          2307, 20057, 12326,  1997,  1996, 26232,  1998,  1037,  3451, 12696,
          2005,  2026,  4245,  1997,  2216,  2040,  3305,  2008,  2512,  1011,
          7473,  2003,  2428,  6057,  1010,  2053,  3043,  2040,  2017,  2024,
           999,  7787,  1998, 12403,  2480,  2024, 20672,  1010,  2004,  2003,
          1996,  2980,  3899,  5983,  5

In [10]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(...)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def compute_metrics(eval_data):
    ...
    
    preds = np.argmax(logits, -1)
    return {
        "f1": metrics.f1_score(labels, preds)
    }

training_args = transformers.TrainingArguments(
    ...
)

In [12]:
trainer = transformers.Trainer(
    ...
)

2024-08-18 04:22:38.690218: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 04:22:38.690288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 04:22:38.691723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.369787,0.872180
2,No log,0.329566,0.880363
3,No log,0.324610,0.877582


TrainOutput(global_step=375, training_loss=0.3730139567057292, metrics={'train_runtime': 168.2729, 'train_samples_per_second': 142.625, 'train_steps_per_second': 2.229, 'total_flos': 249110795520000.0, 'train_loss': 0.3730139567057292, 'epoch': 3.0})

In [14]:
trainer.save_state()

In [15]:
trainer.save_model()